This example illustrates processing tensorflow datasets (IMDB)

In [4]:
import tensorflow as tf
print(tensorflow.__version__)

2.0.0


In [9]:
# run this with tensorflow 1.x version
# tensorflow.enable_eager_execution()

In [10]:
# pip install tensorflow-datasets
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to C:\Users\Miaojunli\tensorflow_datasets\imdb_reviews\plain_text\0.1.0. Subsequent calls will reuse this data.


In [18]:
print(imdb.keys())

dict_keys(['test', 'train', 'unsupervised'])


In [11]:
import numpy as np

training_data, testing_data = imdb['train'],imdb['test']

In [52]:
training_sentences=[]
training_labels=[]

testing_sentences=[]
testing_labels=[]

for s,l in training_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
    
for s,l in testing_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [53]:
print(len(training_sentences))
print(len(testing_sentences))
# print(training_sentences[0], training_labels[0])

25000
25000


In [54]:
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [46]:
vocab_size=10000
embedding_dim=16
max_length=120
trunc_type='post'
oov_tok='<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token=oov_tok, num_words=vocab_size)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [47]:
testing_padded[0]

array([ 153,   13,   93,   12,   18,  315, 4275,   14,    2,    1,    1,
       1061,  234,  439,  163,   19,   13,   24,   62,  149,  719, 2184,
        782,    1,    1,    3,   50,    5,  135, 1082, 1459,   32,  207,
          9,    2,   18,    1,    7,  416,   54, 1031,  402,  111,    4,
        169, 2669,   27,   93,    3,   78,  145,   62,  152,   12,    7,
        239,    5,  189,   18,  565,  406,   68,   52,  293,    9,  209,
        101,    3,   84,    9, 3310,  100,   11, 2155,   92,    5,   23,
         31,  216,    1,    1,    2,   18,    3,  242,  198,  149,  108,
         64,  141,   16,   35,  473,   31,  319, 2168,  136,    7,   23,
        130,  107,   46,   16,    2,  159,  504,  596,   46,   38,    1,
         95, 1001,   69,  410,    2,   52,  157,    1,   77,  119])

In [48]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [55]:
from tensorflow.keras.optimizers import Adam, RMSprop

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])

In [56]:
# model.fit(padded, training_labels_final, epochs=15)
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 3s 135us/sample - loss: 0.5042 - acc: 0.7296 - val_loss: 0.3521 - val_acc: 0.8456
Epoch 2/10
25000/25000 [==============================] - 3s 119us/sample - loss: 0.2417 - acc: 0.9078 - val_loss: 0.3708 - val_acc: 0.8388
Epoch 3/10
25000/25000 [==============================] - 3s 126us/sample - loss: 0.0884 - acc: 0.9788 - val_loss: 0.4683 - val_acc: 0.8244
Epoch 4/10
25000/25000 [==============================] - 3s 120us/sample - loss: 0.0239 - acc: 0.9972 - val_loss: 0.5209 - val_acc: 0.8262
Epoch 5/10
25000/25000 [==============================] - 3s 126us/sample - loss: 0.0071 - acc: 0.9994 - val_loss: 0.5862 - val_acc: 0.8282
Epoch 6/10
25000/25000 [==============================] - 3s 130us/sample - loss: 0.0024 - acc: 0.9999 - val_loss: 0.6340 - val_acc: 0.8288
Epoch 7/10
25000/25000 [==============================] - 3s 126us/sample - loss: 9.9118e-04 - acc: 1.0000 - v

In [57]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [59]:
weights[0]

array([ 0.01786454,  0.00730712, -0.01030541,  0.0076415 , -0.05454065,
       -0.01769597,  0.01719486, -0.0430806 , -0.02071525,  0.12593666,
       -0.04117364,  0.00558874, -0.01396803, -0.00062769,  0.00054454,
       -0.01596864], dtype=float32)

Create weights file and load to projector.tensorflow.org to understand how words are clustered 

In [60]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b oh yeah jenna jameson did it again yeah baby this movie rocks it was one of the 1st movies i saw of her and i have to say i feel in love with her she was great in this move br br her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing br br i hope this comment helps and u can buy the movie the storyline is awesome is very unique and i'm sure u are going to like it jenna amazed us once more and no wonder the movie won so many
b"Oh yeah! Jenna Jameson did it again! Yeah Baby! This movie rocks. It was one of the 1st movies i saw of her. And i have to say i feel in love with her, she was great in this move.<br /><br />Her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing.<br /><br />I hope this comment helps and u can buy the movie, the

In [66]:
type(reverse_word_index)

dict

In [68]:
from itertools import islice 
print(list(islice(reverse_word_index.items(),10)))
print(list(islice(word_index.items(),10)))

[(1, '<OOV>'), (2, 'the'), (3, 'and'), (4, 'a'), (5, 'of'), (6, 'to'), (7, 'is'), (8, 'br'), (9, 'in'), (10, 'it')]
[('<OOV>', 1), ('the', 2), ('and', 3), ('a', 4), ('of', 5), ('to', 6), ('is', 7), ('br', 8), ('in', 9), ('it', 10)]


In [74]:
type(reverse_word_index)
reverse_word_index[1]

'<OOV>'

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

### Adding LSTM

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

from tensorflow.keras.optimizers import Adam, RMSprop

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])

num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))